In [ ]:
# !pip install surprise
# !pip install category_encoders
# !pip install miceforest --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 249.6 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


# Libraries

In [ ]:
%matplotlib inline

from pathlib import Path
import pandas as pd
import numpy as np
from pandas.core.indexes.extension import Index

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

from sklearn.preprocessing import minmax_scale
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder
from copy import deepcopy
import joblib

import miceforest as mf
from sklearn.pipeline import Pipeline

import lightgbm as lgb
import math
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
pd.set_option('display.max_columns', None)

# Connecting to colab

In [ ]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
else:
  print('Not running on CoLab')

Running on CoLab


## Mounting to google drive

In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


# Cleaning Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/WALMART/CAPSTONE project/Data/online_retail_co_ecommerce.csv')
df['number_of_reviews'] = df['number_of_reviews'].str.replace(',','')
df['number_of_reviews'] = df['number_of_reviews'].astype('float')
df[['number_available_in_stock', 'condition']] = df.number_available_in_stock.str.split(expand = True)
df['number_available_in_stock'] = df['number_available_in_stock'].astype('float')
df['price'] = df['price'].str.split().str[0]
df['price'] = df['price'].str.split(' ',1, expand = True)
df['price'] = df['price'].str.replace('£','')
df['price'] = df['price'].str.replace(',','')
df['price'] = df['price'].astype('float')
string_count = df[df['average_review_rating'].astype(str).str.contains('out of 5 stars')]
df['average_review_rating'] = df['average_review_rating'].str.split().str[0]
df['average_review_rating'] = df['average_review_rating'].astype('float')

df['description'] = df['description'].fillna('')
df['new_desc'] = df['product_name'] + df['description']
df['new_desc'] = df['new_desc'].fillna('')

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['new_desc'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

df = df.reset_index()
titles = df[['category_and_sub_category']]
indices = pd.Series(df.index, index= df['uniq_id']) 

def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:15]
    product = [i[0] for i in sim_scores]
    return titles.iloc[product]

for i, row in df.loc[df.category_and_sub_category.isnull()].iterrows():
  if (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[0,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[0,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[1,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[1,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[2,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[2,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[3,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[3,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[4,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[4,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[5,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[5,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[6,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[6,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[7,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[7,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[8,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[8,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[9,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[9,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[10,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[10,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[11,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[11,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[12,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[12,0]
  elif (pd.isna(get_recommendations(df['uniq_id'][i]).iloc[13,0]) == False):
    df['category_and_sub_category'][i] = get_recommendations(df['uniq_id'][i]).iloc[13,0]
  else:
    df['category_and_sub_category'][i] = None

def find_between( s, first, last ):
  start = s.index( first ) + len( first )
  end = s.index( last, start )
  return s[start:end]

df['new'] = '0'
for i, row in df.loc[df.category_and_sub_category.isnull() & (df.product_information.notnull())].iterrows():
  if df['product_information'][i].count('#') != 0:
    df['new'][i]= find_between(df['product_information'][i], "#" , ">")

df['new'] = df.new.str.split('in',1).str[-1]
df['new'] = df['new'].str[1:]

for i, row in df.loc[df.category_and_sub_category.isnull() & (df.product_information.notnull())].iterrows():
  df['category_and_sub_category'][i] = df['new'][i]

df = df.drop(['new'], axis = 1)

cols = np.arange(df.category_and_sub_category.str.split(' > ',expand = True).shape[1])
cols = cols.tolist()
string = 'cat'
cols2 = [str(i) + string for i in cols]
df[cols2] = df.category_and_sub_category.str.split(' > ',expand = True)

df['customers_who_bought_this_item_also_bought'] = df['customers_who_bought_this_item_also_bought'].str.replace(' ','')
df['customers_who_bought_this_item_also_bought'] = df['customers_who_bought_this_item_also_bought'].str.replace('http://www.onlineretail.co/','')
cols = np.arange(df.customers_who_bought_this_item_also_bought.str.split('|',expand = True).shape[1])
cols = cols.tolist()
string = 'together'
cols4 = [str(i) + string for i in cols]
df[cols4] = df.customers_who_bought_this_item_also_bought.str.split('|',expand = True)
df[cols4] = df[cols4].replace('(/).*','', regex = True)
df[cols4] = df[cols4].replace('-',' ', regex = True)

df['items_customers_buy_after_viewing_this_item'] = df['items_customers_buy_after_viewing_this_item'].str.replace(' ','')
df['items_customers_buy_after_viewing_this_item'] = df['items_customers_buy_after_viewing_this_item'].str.replace('http://www.onlineretail.co/','')
cols2 = np.arange(df.items_customers_buy_after_viewing_this_item.str.split('|',expand = True).shape[1])
cols2 = cols2.tolist()
string = 'after'
cols6 = [str(i) + string for i in cols2]
df[cols6] = df.items_customers_buy_after_viewing_this_item.str.split('|',expand = True)
df[cols6] = df[cols6].replace('(/).*','', regex = True)
df[cols6] = df[cols6].replace('-',' ', regex = True)

df['manufacturer'] = df['manufacturer'].replace(np.nan, 'Unknown')

df['new'] = 0
for i, row in df.loc[df.price.isnull() & (df.sellers.notnull())].iterrows():
  df['new'][i]= find_between(df['sellers'][i], "£","}")

df['new'] = df['new'].str.replace('"','')
df['new'] = df['new'].str.replace(',','')
df['new']= df['new'].str.split().str[0]
df['new'] = df['new'].astype('float')

for i, row in df.loc[df.price.isnull() & (df.sellers.notnull())].iterrows():
  df['price'][i] = df['new'][i]

df = df.drop(['new'], axis = 1)

df['number_of_reviews'] = df['number_of_reviews'].replace(np.nan, 0)

df['number_available_in_stock'] = df['number_available_in_stock'].replace(np.nan, 0)

df['average_review_rating'] = df['average_review_rating'].replace(np.nan, 0)

df['condition'] = df['condition'].replace(np.nan, 'not_avaiable')

df['new'] = '0'
for i, row in df.loc[df.number_of_answered_questions.isnull() & (df.customer_questions_and_answers.notnull())].iterrows():
  df['new'][i] = df['customer_questions_and_answers'][i].count('//')

df['new'] = df['new'].astype('float')

for i, row in df.loc[df.number_of_answered_questions.isnull() & (df.customer_questions_and_answers.notnull())].iterrows():
  df['number_of_answered_questions'][i] = df['new'][i]

df = df.drop(['new'], axis = 1)

df['number_of_answered_questions'] = df['number_of_answered_questions'].replace(np.nan, 0)

df['number_of_sellers'] = '0'
for row in df.loc[df.sellers.notnull()].iterrows():
  df['number_of_sellers'] = df['sellers'].str.count('Seller_name')

df['number_of_sellers'] = df['number_of_sellers'].astype('float')

df['number_of_sellers'] = df['number_of_sellers'].replace(np.nan, 0)

scaler = minmax_scale
df['number_of_reviews'] = df['number_of_reviews'].replace(0,1)
df['scaled_reviews_by_category_log'] = df.groupby('0cat').number_of_reviews.transform(lambda x: np.log10(x.astype(float)))
df['scaled_reviews_by_category_after_log'] = df.groupby('0cat').scaled_reviews_by_category_log.transform(lambda x: minmax_scale(x.astype(float)))
df = df.drop(['scaled_reviews_by_category_log'], axis = 1)
df['popularity'] = df['scaled_reviews_by_category_after_log'] * df['average_review_rating']

y = df['number_of_reviews']
X = df.drop(['number_of_reviews'], axis = 1)

df3 = X[['uniq_id', 'manufacturer', 'price', 'number_available_in_stock', 'number_of_answered_questions',\
          'average_review_rating', 'category_and_sub_category', 'condition', 'number_of_sellers']].copy()

df3.set_index(['uniq_id'], inplace = True)

cols_to_drop = []
cat_encoders = {}
cat_enc_columns = []

for col in df3.columns:
    if df3[col].dtype == 'object':
        if df3[col].nunique() < 20:
            enc = OneHotEncoder( handle_unknown='ignore', sparse=False)
            enc.fit(df3[[col]])
            joblib.dump(enc, col + '_enc.joblib')
            result = enc.transform(df3[[col]])
            ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
            cat_enc_columns = cat_enc_columns + ohe_columns
            result_train = pd.DataFrame(result, columns=ohe_columns , index=df3.index)
            df3 = pd.concat([df3, result_train.reindex(df3.index)], axis=1 , join='inner')

        else:
            te=TargetEncoder(handle_missing='value',handle_unknown='value', min_samples_leaf=5, smoothing=0.1)
            te.fit(df3[col], y.values)
            df3[col+'_te']=te.transform(df3[col])
            cat_encoders[col] = [deepcopy(te),"te"]
            cat_enc_columns.append(col+'_te')
        cols_to_drop.append(col)

df3.drop(cols_to_drop, axis=1, inplace=True)

kernel = mf.ImputationKernel(df3, datasets=1 )

pipe = Pipeline([('impute', kernel)])
X_t = pipe.fit_transform(df3, y, impute__iterations=3)
X_t['uniq_id'] = X_t.index
df['price'] = X_t['price'].values
df = df.drop(['index'], axis = 1)
df = df.drop(['Unnamed: 0'], axis = 1)
df = df.drop(['scaled_reviews_by_category_after_log'], axis = 1)

In [ ]:
# df.to_csv('/content/drive/MyDrive/WALMART/CAPSTONE project/Data/Temp data/df_after_clean_updated.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 42 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      10000 non-null  object 
 1   product_name                                 10000 non-null  object 
 2   manufacturer                                 10000 non-null  object 
 3   price                                        10000 non-null  float64
 4   number_available_in_stock                    10000 non-null  float64
 5   number_of_reviews                            10000 non-null  float64
 6   number_of_answered_questions                 10000 non-null  float64
 7   average_review_rating                        10000 non-null  float64
 8   category_and_sub_category                    10000 non-null  object 
 9   customers_who_bought_this_item_also_bought   8938 non-null   object 
 10 

# Recommendation Function

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/WALMART/CAPSTONE project/Data/Temp data/df_after_clean.csv')

In [ ]:

def recommender(unique_id, manufacturer1, manufacturer2):
  


  """Product Name"""

  a = df[df['uniq_id'] == unique_id]['product_name'].values[0]
  aa = "Product's name: " + a



  """Most frequent item bought after viewing this product"""

  index = np.int((df.loc[df['uniq_id'].isin([unique_id])].index)[0])
  if pd.isna(df['0after'].iloc[index]) == False:
    b = (df[cols6].iloc[index , :].mode())[0]
    bb = "Most frequent item bought after viewing this product: " + b
  else:
    base5 = pd.DataFrame(df.loc[df['uniq_id'].isin([unique_id])])
    base55 = df[df['0after'].notnull()]
    temp5 = pd.concat([base5, base55])
    temp5 = temp5.reset_index(drop = True)
    temp5['new_desc'] = temp5['new_desc'].fillna('')
    tf5 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    tfidf_matrix5 = tf5.fit_transform(temp5['new_desc'])
    cosine_sim5 = linear_kernel(tfidf_matrix5, tfidf_matrix5)


    titles5 = temp5[['uniq_id']]
    indices5 = pd.Series(temp5.index, index= temp5['uniq_id'])
    def get_recommendations5(title5):
      idx5 = indices5[title5]
      sim_scores5 = list(enumerate(cosine_sim5[idx5]))
      sim_scores5 = sorted(sim_scores5, key=lambda x: x[1], reverse=True)
      sim_scores5 = sim_scores5[1:2]
      product5 = [i[0] for i in sim_scores5]
      return titles5.iloc[product5]
    sim_p_id = get_recommendations5(unique_id).iloc[0,0]
    index_s = np.int((df.loc[df['uniq_id'].isin([sim_p_id])].index)[0])
    b = (df[cols6].iloc[index_s , :].mode())[0]
    bb = "Most frequent item bought after viewing this product(similar): " + b



  """Most frequent item bought after buying this product"""

  if pd.isna(df['0together'].iloc[index]) == False:
    c = (df[cols4].iloc[index , :].mode())[0]
    cc = "Most frequent item bought after buying this product: " + c
  else:
    base6 = pd.DataFrame(df.loc[df['uniq_id'].isin([unique_id])])
    base66 = df[df['0together'].notnull()]
    temp6 = pd.concat([base6, base66])
    temp6 = temp6.reset_index(drop = True)
    temp6['new_desc'] = temp6['new_desc'].fillna('')
    tf6 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    tfidf_matrix6 = tf6.fit_transform(temp6['new_desc'])
    cosine_sim6 = linear_kernel(tfidf_matrix6, tfidf_matrix6)


    titles6 = temp6[['uniq_id']]
    indices6 = pd.Series(temp6.index, index= temp6['uniq_id'])
    def get_recommendations6(title6):
      idx6 = indices6[title6]
      sim_scores6 = list(enumerate(cosine_sim6[idx6]))
      sim_scores6 = sorted(sim_scores6, key=lambda x: x[1], reverse=True)
      sim_scores6 = sim_scores6[1:2]
      product6 = [i[0] for i in sim_scores6]
      return titles6.iloc[product6]
    sim_p_id_t = get_recommendations6(unique_id).iloc[0,0]
    index_s_t = np.int((df.loc[df['uniq_id'].isin([sim_p_id_t])].index)[0])
    c = (df[cols4].iloc[index_s_t , :].mode())[0]
    cc = "Most frequent item bought after buying this product(similar): " + c 



  """Most popular item in same category and sub category"""
  
  category = df['category_and_sub_category'][df.loc[df['uniq_id'].isin([unique_id])].index[0]]
  temp = pd.DataFrame(df.loc[df['category_and_sub_category'].isin([category])])
  temp = temp[temp['number_available_in_stock'] > 0]

  if temp.empty == False:
    d = temp['product_name'].loc[temp['popularity'].idxmax()]
    dd = "Most popular product in same category: " + d
  else:
    category88 = df['0cat'][df.loc[df['uniq_id'].isin([unique_id])].index[0]]
    temp88 = pd.DataFrame(df.loc[df['0cat'].isin([category88])])
    temp88 = temp88[temp88['number_available_in_stock'] > 0]
    d = temp88['product_name'].loc[temp88['popularity'].idxmax()]
    dd = "Most popular product in Main category: " + d

  """Most similar product based on description"""

  base111 = df[df['number_available_in_stock'] > 0] 
  base222 = df.loc[df['uniq_id'] == unique_id]
  temp22 = pd.concat([base111, base222]).drop_duplicates()
  temp22 = temp22.reset_index(drop = True)
  temp22['new_desc'] = temp22['new_desc'].fillna('')
  tf22 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
  tfidf_matrix22 = tf22.fit_transform(temp22['new_desc'])
  cosine_sim22 = linear_kernel(tfidf_matrix22, tfidf_matrix22)


  titles22 = temp22[['product_name']]
  indices22 = pd.Series(temp22.index, index= temp22['uniq_id'])
  def get_recommendations22(title22):
    idx22 = indices22[title22]
    sim_scores22 = list(enumerate(cosine_sim22[idx22]))
    sim_scores22 = sorted(sim_scores22, key=lambda x: x[1], reverse=True)
    sim_scores22 = sim_scores22[1:3]
    product22 = [i[0] for i in sim_scores22]
    return titles22.iloc[product22]
  
  e = get_recommendations22(unique_id).iloc[0,0]
  ee = "Most similar product to this item is: " + e



  """Similar products from same manufacturer"""

  manufac = df['manufacturer'][df.loc[df['uniq_id'].isin([unique_id])].index[0]]
  base112 = pd.DataFrame(df.loc[df['manufacturer'].isin([manufac])])
  base113 = base112[base112['number_available_in_stock'] > 0] 
  temp2 = pd.concat([base113, base112]).drop_duplicates()
  temp2 = temp2.reset_index(drop = True)
  temp2['new_desc'] = temp2['new_desc'].fillna('')
  tf2 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
  tfidf_matrix2 = tf2.fit_transform(temp2['new_desc'])
  cosine_sim2 = linear_kernel(tfidf_matrix2, tfidf_matrix2)


  titles2 = temp2[['product_name']]
  indices2 = pd.Series(temp2.index, index= temp2['uniq_id'])
  def get_recommendations2(title2):
    idx2 = indices2[title2]
    sim_scores2 = list(enumerate(cosine_sim2[idx2]))
    sim_scores2 = sorted(sim_scores2, key=lambda x: x[1], reverse=True)
    sim_scores2 = sim_scores2[1:6]
    product2 = [i[0] for i in sim_scores2]
    return titles2.iloc[product2]
  mm = []
  for i in range(0,len(get_recommendations2(unique_id))):
    mm[i] = mm.append(i)
    mm[i] = get_recommendations2(unique_id).iloc[i,0]



    """Similar products from promoted manufacturer #1"""

  base1 = pd.DataFrame(df.loc[df['manufacturer'].isin([manufacturer1])])
  base1 = base1[base1['number_available_in_stock'] > 0]
  base2 = df.loc[df['uniq_id'] == unique_id]
  temp3 = pd.concat([base1, base2])

  if temp3.empty == False:
    temp3 = temp3.reset_index(drop = True)
    temp3['new_desc'] = temp3['new_desc'].fillna('')
    tf3 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    tfidf_matrix3 = tf3.fit_transform(temp3['new_desc'])
    cosine_sim3 = linear_kernel(tfidf_matrix3, tfidf_matrix3)


    titles3 = temp3[['product_name']]
    indices3 = pd.Series(temp3.index, index= temp3['uniq_id'])
    def get_recommendations3(title3):
      idx3 = indices3[title3]
      sim_scores3 = list(enumerate(cosine_sim3[idx3]))
      sim_scores3 = sorted(sim_scores3, key=lambda x: x[1], reverse=True)
      sim_scores3 = sim_scores3[1:3]
      product3 = [i[0] for i in sim_scores3]
      return titles3.iloc[product3]
    mm3 = []
    for i in range(0,len(get_recommendations3(unique_id))):
      mm3[i] = mm3.append(i)
      mm3[i] = get_recommendations3(unique_id).iloc[i,0]
  else:
    mm3 ='"{}" has no available products'.format(manufacturer1)


  
    """Similar products from promoted manufacturer #2"""

  base11 = pd.DataFrame(df.loc[df['manufacturer'].isin([manufacturer2])])
  base11 = base11[base11['number_available_in_stock'] > 0]
  base22 = df.loc[df['uniq_id'] == unique_id]
  temp4 = pd.concat([base11, base22])

  if temp4.empty == False:
    temp4 = temp4.reset_index(drop = True)
    temp4['new_desc'] = temp4['new_desc'].fillna('')
    tf4 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    tfidf_matrix4 = tf4.fit_transform(temp4['new_desc'])
    cosine_sim4 = linear_kernel(tfidf_matrix4, tfidf_matrix4)


    titles4 = temp4[['product_name']]
    indices4 = pd.Series(temp4.index, index= temp4['uniq_id'])
    def get_recommendations4(title4):
      idx4 = indices4[title4]
      sim_scores4 = list(enumerate(cosine_sim4[idx4]))
      sim_scores4 = sorted(sim_scores4, key=lambda x: x[1], reverse=True)
      sim_scores4 = sim_scores4[1:3]
      product4 = [i[0] for i in sim_scores4]
      return titles4.iloc[product4]
    mm4 = []
    for i in range(0,len(get_recommendations4(unique_id))):
      mm4[i] = mm4.append(i)
      mm4[i] = get_recommendations4(unique_id).iloc[i,0]

  else:
    mm4 ='"{}" has no available products'.format(manufacturer2)


  
  return aa, bb, cc, dd, ee,'Other similar products that "{}" produces are:'.format(manufac), mm,\
  'Other similar products that "{}" produces are:'.format(manufacturer1), mm3,\
  'Other similar products that "{}" produces are:'.format(manufacturer2), mm4

In [ ]:
recommender('85e27ae7027190252fd4f5e3fde85eb0', 'Turtles', 'Halo')

("Product's name: Minecraft Figures (Pack of 3)",
 'Most frequent item bought after viewing this product: Minecraft Core Animal Mob Pack',
 'Most frequent item bought after buying this product: John Adams Moshi Monsters Waggle',
 'Most popular product in same category: Playmobil 5494 Christmas Advent Calendar Santas Workshop',
 'Most similar product to this item is: Minecraft Cow/Steve with Pickaxe and Spider Figures (Pack of 3)',
 'Other similar products that "Minecraft" produces are:',
 ['Minecraft Cow/Steve with Pickaxe and Spider Figures (Pack of 3)',
  'Minecraft Core Animal Mob (Pack of 6)',
  'Minecraft 3-inch Alex Figure with Accessory',
  'Minecraft Steve ~6" Vinyl Figure'],
 'Other similar products that "Turtles" produces are:',
 ['Turtles Raph with Mini Cycle Half-Shell Heroes (Pack of 2)',
  'Turtles Shredder and Foot Soldier Half-Shell Heroes (Pack of 2)'],
 'Other similar products that "Halo" produces are:',
 ['HALO 5 Guardians Series 1 Spartan Variant Action Figure',
  '

# from here is just to test